<a href="https://colab.research.google.com/github/KSY1526/myblog/blob/master/_notebooks/2021-12-24-Do_natural_language5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# "[Do it 자연어] 5. 개체명 인식하기 + 웹 실습"
- author: Seong Yeon Kim 
- categories: [book, jupyter, Do it, natural language, BERT, tokenizer, web, Classifier]

# 모델 환경설정

In [1]:
!pip install ratsnlp

     |████████████████████████████████| 53 kB 807 kB/s 
     |████████████████████████████████| 57 kB 2.5 MB/s 
     |████████████████████████████████| 2.8 MB 9.7 MB/s 
     |████████████████████████████████| 806 kB 11.1 MB/s 
     |████████████████████████████████| 132 kB 36.0 MB/s 
     |████████████████████████████████| 332 kB 45.3 MB/s 
     |████████████████████████████████| 636 kB 39.2 MB/s 
     |████████████████████████████████| 829 kB 19.7 MB/s 
     |████████████████████████████████| 3.3 MB 33.1 MB/s 
     |████████████████████████████████| 895 kB 45.4 MB/s 
     |████████████████████████████████| 61 kB 501 kB/s 
     |████████████████████████████████| 1.1 MB 41.0 MB/s 
     |████████████████████████████████| 96 kB 4.5 MB/s 
     |████████████████████████████████| 160 kB 44.3 MB/s 
     |████████████████████████████████| 192 kB 43.9 MB/s 
     |████████████████████████████████| 271 kB 43.7 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 s

In [2]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [3]:
from torch.cuda import is_available
import torch
from ratsnlp.nlpbook.ner import NERTrainArguments

args = NERTrainArguments(
    pretrained_model_name = 'beomi/kcbert-base',
    downstream_corpus_name = 'ner', # 한국해양대학교 자연언어처리연구실 데이터 + 자체데이터
    downstream_model_dir = '/gdrive/My Drive/nlpbook/checkpoint-ner', # 모델 체크포인트 저장위치
    batch_size = 32 if torch.cuda.is_available() else 4,
    learning_rate = 5e-5,
    max_seq_length = 64,
    epochs = 3,
    tpu_cores = 0 if torch.cuda.is_available() else 8,
    seed = 7,
)

앞서 한것과 크게 다르지 않습니다. 프리 트레인 모델로 이준범님의 kcbert-base 모델을 사용했습니다.

ner, 한국해양대학교 데이터를 실습 데이터로 사용하겠습니다.

In [4]:
from ratsnlp import nlpbook
nlpbook.set_seed(args)
nlpbook.set_logger(args)

INFO:ratsnlp:Training/evaluation parameters NERTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='named-entity-recognition', downstream_corpus_name='ner', downstream_corpus_root_dir='/root/Korpora', downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-ner', max_seq_length=64, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=3, batch_size=32, cpu_workers=2, fp16=False, tpu_cores=0)


set seed: 7


랜덤 시드를 고정했으며 로거를 설정했습니다.

# 말뭉치 내려받기

In [5]:
nlpbook.download_downstream_dataset(args)

Downloading: 100%|██████████| 17.9M/17.9M [00:00<00:00, 33.8MB/s]
Downloading: 100%|██████████| 1.13M/1.13M [00:00<00:00, 27.2MB/s]


말뭉치를 내려받습니다.

In [6]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case = False,
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

토크나이저를 선언합니다.

# 학습 데이터 구축

In [7]:
from ratsnlp.nlpbook.ner import NERCorpus, NERDataset
corpus = NERCorpus(args)
train_dataset = NERDataset(
    args = args,
    corpus = corpus,
    tokenizer = tokenizer,
    mode = 'train'
)

INFO:ratsnlp:Creating features from dataset file at /root/Korpora/ner
INFO:ratsnlp:loading train data... LOOKING AT /root/Korpora/ner/train.txt
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence: 이어 옆으로 움직여 김일성의 오른쪽에서 한 차례씩 두 번 상체를 굽혀 조문했으며 이윽고 안경을 벗고 손수건으로 눈주위를 닦기도 했다.
INFO:ratsnlp:target: 이어 옆으로 움직여 <김일성:PER>의 오른쪽에서 <한 차례:NOH>씩 <두 번:NOH> 상체를 굽혀 조문했으며 이윽고 안경을 벗고 손수건으로 눈주위를 닦기도 했다.

INFO:ratsnlp:tokens: [CLS] 이어 옆 ##으로 움직 ##여 김일성 ##의 오른 ##쪽에서 한 차례 ##씩 두 번 상 ##체를 굽 ##혀 조문 ##했 ##으며 이 ##윽 ##고 안 ##경을 벗고 손 ##수 ##건으로 눈 ##주 ##위를 닦 ##기도 했다 . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
INFO:ratsnlp:label: [CLS] O O O O O B-PER O O O B-NOH I-NOH O B-NOH I-NOH O O O O O O O O O O O O O O O O O O O O O O O [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
INFO:ratsnlp:features: NERFe

NERCorpus가 넘겨준 데이터와 kcbert-base 데이터로 학습한 토크나이저를 사용해 4가지 출력값을 줍니다.

마지막 출력값은 label_ids로 B-PER(시작-인명), I-PER(시작아님-인명) B-NOH(시작-기타수량표현) 등을 정수를 할당해 표현합니다. 

In [8]:
from torch.utils.data import DataLoader, RandomSampler
train_dataloader = DataLoader(
    train_dataset,
    batch_size = args.batch_size,
    sampler = RandomSampler(train_dataset, replacement = False),
    collate_fn = nlpbook.data_collator,
    drop_last = False,
    num_workers = args.cpu_workers,
)

학습 데이터 셋으로 로더를 구축했습니다. 배치크기만큼 인스턴스를 랜덤하게 뽑은 뒤 이를 합처 배치를 만듭니다.

# 평가 데이터 구축

In [9]:
from torch.utils.data import SequentialSampler

val_dataset = NERDataset(
    args = args,
    corpus = corpus,
    tokenizer = tokenizer,
    mode = 'val',
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size = args.batch_size,
    sampler = SequentialSampler(val_dataset), # 순서대로 추출
    collate_fn = nlpbook.data_collator,
    drop_last = False,
    num_workers = args.cpu_workers,
)

INFO:ratsnlp:Creating features from dataset file at /root/Korpora/ner
INFO:ratsnlp:loading val data... LOOKING AT /root/Korpora/ner/val.txt
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence: 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다
INFO:ratsnlp:target: 결국 초연은 <4년 반:DUR>이 지난 후에 <드레스덴:LOC>에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다

INFO:ratsnlp:tokens: [CLS] 결국 초 ##연 ##은 4년 반 ##이 지난 후에 드 ##레스 ##덴 ##에서 연 ##주 ##되었고 재 ##연 ##도 이루어 ##졌지 ##만 , 이후에 그대로 방치 ##되고 말았 ##다 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
INFO:ratsnlp:label: [CLS] O O O O B-DUR I-DUR O O O B-LOC I-LOC I-LOC O O O O O O O O O O O O O O O O O [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
INFO:rats

입력받은 데이터를 토크나이저로 분리한 후 로더를 이용해 테스트용 배치를 만듭니다.

# 모델 학습

In [10]:
from transformers import BertConfig, BertForTokenClassification

pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels = corpus.num_labels,
)
# 프리트레인 BERT 모델

model = BertForTokenClassification.from_pretrained(
    args.pretrained_model_name,
    config = pretrained_model_config,
)
# 프리트레인 모델 + 개체명 인식을 위한 태스크모듈

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the

윗 코드는 BERT 모델을 프리트레인 했고, 아랫 코드는 개체명 인식을 위한 태스크 모듈을 덧붙힌 모델입니다.

In [11]:
from ratsnlp.nlpbook.ner import NERTask
task = NERTask(model, args)
trainer = nlpbook.get_trainer(args)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: Checkpoint directory /gdrive/My Drive/nlpbook/checkpoint-ner exists and is not empty.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores


앞서 만든 모델을 이용해 데이터를 학습시킵니다. NERTask는 아담 옵티마이저와 ExponentialLR 러닝레이트를 사용합니다.

그 뒤 GPU/TPU 설정, 로그 및 체크포인트등을 하는 트레이너를 정의합니다.

In [12]:
trainer.fit(
    task,
    train_dataloader = train_dataloader,
    val_dataloaders = val_dataloader,
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | BertForTokenClassification | 108 M 
-----------------------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
433.389   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

모델을 학습합니다.

# 개체명 인식 웹 서비스 만들기

In [15]:
from ratsnlp.nlpbook.ner import NERDeployArguments
args = NERDeployArguments(
    pretrained_model_name= 'beomi/kcbert-base',
    downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-ner',
    max_seq_length=64,
)

downstream_model_checkpoint_fpath: /gdrive/My Drive/nlpbook/checkpoint-ner/epoch=1-val_loss=0.20.ckpt
downstream_model_labelmap_fpath: /gdrive/My Drive/nlpbook/checkpoint-ner/label_map.txt


인퍼러스 설정을 다시 해줍니다.

In [16]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case = False,
)

토크나이저도 초기화해줍니다.

In [20]:
import torch
from transformers import BertConfig, BertForTokenClassification
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location = torch.device('cpu'),
)

pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels = fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel(),
)

model = BertForTokenClassification(pretrained_model_config)

체크포인트 로드, BERT 설정 로드, BERT 모델 초기화를 했습니다.

In [21]:
model.load_state_dict({k.replace('model.', ''): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})
model.eval()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

체크포인트를 주입하고 모델을 평가모드로 바꿉니다.

In [22]:
labels = [label.strip() for label in open(args.downstream_model_labelmap_fpath, "r").readlines()]
id_to_label = {}
for idx, label in enumerate(labels):
  if "PER" in label:
    label = "인명"
  elif "LOC" in label:
    label = "지명"
  elif "ORG" in label:
    label = "기관명"
  elif "DAT" in label:
    label = "날짜"
  elif "TIM" in label:
    label = "시간"
  elif "DUR" in label:
    label = "기간"
  elif "MNY" in label:
    label = "통화"
  elif "PNT" in label:
    label = "비율"
  elif "NOH" in label:
    label = "기타 수량표현"
  elif "POH" in label:
    label = "기타"
  else:
    label = label
  id_to_label[idx] = label

id_to_label

{0: '[CLS]',
 1: '[SEP]',
 2: '[PAD]',
 3: '[MASK]',
 4: 'O',
 5: '인명',
 6: '기타 수량표현',
 7: '기타',
 8: '기관명',
 9: '날짜',
 10: '지명',
 11: '통화',
 12: '비율',
 13: '시간',
 14: '기간',
 15: '인명',
 16: '기타 수량표현',
 17: '기타',
 18: '기관명',
 19: '날짜',
 20: '지명',
 21: '통화',
 22: '비율',
 23: '시간',
 24: '기간'}

정수 인덱스를 레이블에 매핑하는 사전을 만듭니다.

In [23]:
def inference_fn(sentence):
    inputs = tokenizer(
        [sentence],
        max_length = args.mex_seq_length,
        padding = 'max_length',
        truncation = True,
    )

    with torch.no_grad():
        outputs = model(**{k : torch.tenser(v) for k, v in inputs.items()}),
        probs = outputs.logits[0].softmax(dim = 1),
        # 로짓에 소프트맥스를 취해 토큰이 어떤 개체명에 들어가나 확률 구하기.
        top_probs, preds = torch.topk(probs, dim = 1, k = 1),
        # 가장 높은 확률 값과 그 개체 인덱스 구하기
        tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0]),
        # 토큰화된 것을 다시 읽을 수 있는 문자(한국어)로 복원
        predicted_tags = [id_to_label[pred.item()] for pred in preds]
        # 개체 인덱스를 전에 라벨링한 문자로 바꿔주기.(PER => 인명)
        result = []
        
        for token, predicted_tag, top_prob in zip(tokens, predicted_tags, top_probs):
            if token not in [tokenizer.pad_token, tokenizer.cls_token,tokenizer.seq_token]:
                # 이프문은 CLS, SEP, PAD를 제외하는 역활을 함.
                token_result = {
                    'token': token, # 단어
                    'predicted_tag': predicted_tag, # 개체명
                    'top_prob' : str(round(top_prob[0].item(), 4)), # 그 확률
                }
                result.append(token_result)

        return{'sentence' : sentence, 'result' : result,}

실제 입력값이 들어왓을때 처리 과정에 대한 함수입니다. 코드마다 설명을 상세히 했습니다.

In [24]:
from ratsnlp.nlpbook.ner import get_web_service_app
app = get_web_service_app(inference_fn)
app.run()

 * Serving Flask app "ratsnlp.nlpbook.ner.deploy" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://3f00-35-202-140-61.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [24/Dec/2021 14:57:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Dec/2021 14:57:07] "GET /favicon.ico HTTP/1.1" 404 -


웹 서비스로 구현합니다.

(이부분은 앞코드와 마찬가지로 따로 더 공부해야 오류없이 구동될것 같아요.